In [88]:
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pandas as pd
import sklearn

In [89]:
%store -r cook_county_gdf_quarterly
%store -r CC_nodeaths
cook_county_gdf = cook_county_gdf_quarterly

In [90]:
#add in missing 58 tracts
cook_county_gdf['deaths'] = 1
cook_county_gdf = pd.concat([cook_county_gdf, CC_nodeaths], ignore_index=True)
cook_county_gdf['quarter'].fillna(9999, inplace=True)
cook_county_gdf['year'].fillna(9999, inplace=True)

In [91]:
cook_county_gdf.loc[cook_county_gdf['year'] == 9999, 'GEOID'].nunique()

58

In [158]:
columns_to_keep = ['year', 'STATEFP', 'COUNTYFP', 'TRACTCE','NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER',
       'INTPTLAT', 'INTPTLON', 'geometry']

gdf_quarter = cook_county_gdf.groupby(['GEOID', 'quarter']).agg({'deaths': 'sum', **{col: 'first' for col in columns_to_keep}}).reset_index()

In [159]:
gdf_quarter

,GEOID,quarter,deaths,year,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,17031010100,1.0,5,2017.0,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,POINT (-87.66805 42.02216)
1,17031010100,2.0,6,2016.0,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,POINT (-87.67142 42.02143)
2,17031010100,3.0,7,2019.0,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,POINT (-87.67334 42.02293)
3,17031010100,4.0,6,2016.0,17,031,010100,101,Census Tract 101,G5020,S,379511,0,+42.0212553,-087.6698301,POINT (-87.66686 42.02280)
4,17031010201,1.0,4,2021.0,17,031,010201,102.01,Census Tract 102.01,G5020,S,504464,0,+42.0160077,-087.6801484,POINT (-87.68122 42.01311)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3880,17031980000,1.0,4,2019.0,17,031,980000,9800,Census Tract 9800,G5020,S,19890200,92402,+41.9794191,-087.9024376,POINT (-87.90398 41.97572)
3881,17031980000,2.0,6,2018.0,17,031,980000,9800,Census Tract 9800,G5020,S,19890200,92402,+41.9794191,-087.9024376,POINT (-87.90398 41.97572)
3882,17031980000,4.0,5,2016.0,17,031,980000,9800,Census Tract 9800,G5020,S,19890200,92402,+41.9794191,-087.9024376,POINT (-87.90647 41.97684)
3883,17031980100,4.0,2,2018.0,17,031,980100,9801,Census Tract 9801,G5020,S,2981781,0,+41.7859813,-087.7509001,POINT (-87.74137 41.78952)


### Making DF square - one quarter for each tract


In [160]:
gdf_quarter.loc[gdf_quarter['year'] == 9999, 'deaths'] = 0
gdf_quarter.loc[gdf_quarter['quarter'] == 9999, 'deaths'] = 0

# Replace 9999 with 2014 in the 'year' column
gdf_quarter['year'].replace(9999, 2014, inplace=True)
gdf_quarter['quarter'].replace(9999, 1, inplace=True)

In [161]:
years = range(2014, 2024)
quarters = range(1, 5)
updated_rows = []

stop = 0
for tract in gdf_quarter['GEOID'].unique():
    if stop == 100:
         continue
    existing_years = set(gdf_quarter[gdf_quarter['GEOID'] == tract]['year'])
    existing_quarters = set(gdf_quarter[(gdf_quarter['GEOID'] == tract) & (gdf_quarter['year'].isin(existing_years))]['quarter'])
    missing_years = set(years) - existing_years
    
    
    for year in missing_years:
        missing_quarters = set(quarters) - existing_quarters
        
        for quarter in missing_quarters:
            new_row = {'GEOID': tract, 'year': year, 'quarter': quarter, 'deaths': 0}

            above_rows = gdf_quarter[(gdf_quarter['GEOID'] == tract) & ((gdf_quarter['year'] < year) | ((gdf_quarter['year'] == year) & (gdf_quarter['quarter'] < quarter)))]
    
            if len(above_rows) > 0:
                above_row = above_rows.sort_values(['year', 'quarter']).tail(1)

                for col in columns_to_keep:
                    new_row[col] = above_row[col].values[0]


            updated_rows.append(new_row)
            print(updated_rows)
        
            
    stop += 1

gdf_quarter = pd.concat([gdf_quarter, pd.DataFrame(updated_rows)], ignore_index=False)


[{'GEOID': '17031010400', 'year': 2018.0, 'quarter': 2, 'deaths': 0, 'STATEFP': '17', 'COUNTYFP': '031', 'TRACTCE': '010400', 'NAME': '104', 'NAMELSAD': 'Census Tract 104', 'MTFCC': 'G5020', 'FUNCSTAT': 'S', 'ALAND': 548761, 'AWATER': 602605, 'INTPTLAT': '+42.0064109', 'INTPTLON': '-087.6588157', 'geometry': <POINT (-87.659 42.008)>}]
[{'GEOID': '17031010400', 'year': 2018.0, 'quarter': 2, 'deaths': 0, 'STATEFP': '17', 'COUNTYFP': '031', 'TRACTCE': '010400', 'NAME': '104', 'NAMELSAD': 'Census Tract 104', 'MTFCC': 'G5020', 'FUNCSTAT': 'S', 'ALAND': 548761, 'AWATER': 602605, 'INTPTLAT': '+42.0064109', 'INTPTLON': '-087.6588157', 'geometry': <POINT (-87.659 42.008)>}, {'GEOID': '17031010400', 'year': 2018.0, 'quarter': 2, 'deaths': 0, 'STATEFP': '17', 'COUNTYFP': '031', 'TRACTCE': '010400', 'NAME': '104', 'NAMELSAD': 'Census Tract 104', 'MTFCC': 'G5020', 'FUNCSTAT': 'S', 'ALAND': 548761, 'AWATER': 602605, 'INTPTLAT': '+42.0064109', 'INTPTLON': '-087.6588157', 'geometry': <POINT (-87.659 4

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [163]:
gdf_quarter = gdf_quarter.sort_values(['GEOID', 'year', 'quarter'])

In [164]:
gdf_quarter.head(20)

,GEOID,quarter,deaths,year,STATEFP,COUNTYFP,TRACTCE,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
1,17031010100,2.0,6,2016.0,17,031,010100,101,Census Tract 101,G5020,S,379511.0,0.0,+42.0212553,-087.6698301,POINT (-87.6714206343906 42.021433935507915)
3,17031010100,4.0,6,2016.0,17,031,010100,101,Census Tract 101,G5020,S,379511.0,0.0,+42.0212553,-087.6698301,POINT (-87.6668586545441 42.022804888271565)
0,17031010100,1.0,5,2017.0,17,031,010100,101,Census Tract 101,G5020,S,379511.0,0.0,+42.0212553,-087.6698301,POINT (-87.66804697796988 42.02215989618944)
2,17031010100,3.0,7,2019.0,17,031,010100,101,Census Tract 101,G5020,S,379511.0,0.0,+42.0212553,-087.6698301,POINT (-87.6733444534884 42.02292963176082)
5,17031010201,2.0,3,2018.0,17,031,010201,102.01,Census Tract 102.01,G5020,S,504464.0,0.0,+42.0160077,-087.6801484,POINT (-87.68124323315118 42.01854918668072)
6,17031010201,3.0,5,2019.0,17,031,010201,102.01,Census Tract 102.01,G5020,S,504464.0,0.0,+42.0160077,-087.6801484,POINT (-87.68370064493776 42.01716271104117)
7,17031010201,4.0,2,2020.0,17,031,010201,102.01,Census Tract 102.01,G5020,S,504464.0,0.0,+42.0160077,-087.6801484,POINT (-87.68358096135202 42.01597468223533)
4,17031010201,1.0,4,2021.0,17,031,010201,102.01,Census Tract 102.01,G5020,S,504464.0,0.0,+42.0160077,-087.6801484,POINT (-87.68121603961377 42.01310882671215)
10,17031010202,3.0,4,2017.0,17,031,010202,102.02,Census Tract 102.02,G5020,S,351253.0,0.0,+42.0160476,-087.6733256,POINT (-87.67139582654742 42.01334524902559)
8,17031010202,1.0,2,2018.0,17,031,010202,102.02,Census Tract 102.02,G5020,S,351253.0,0.0,+42.0160476,-087.6733256,POINT (-87.67301729545724 42.01905058323299)


In [148]:
gdf_quarter.shape

(15866, 16)

In [149]:
gdf_quarter['GEOID'].nunique()

1332

In [150]:
# missing_quarters = []

# for tract_geoID in gdf_quarter['GEOID'].unique():
#     for  quarter in all_quarters:
#         if not ((gdf_quarter['GEOID'] == tract_geoID) & (gdf_quarter['year'] == year) & (gdf_quarter['quarter'] == quarter)).any():
#             missing_quarters.append((tract_geoID, year, quarter))

# if missing_quarters:
#     print("The following census tracts are missing at least one quarter between 2014 and 2023:")
#     for missing in missing_quarters:
#         print(" GEOID:", missing[0], "- Year:", missing[1], "- Quarter:", missing[2])
# else:
#     print("All census tracts have data for every quarter between 2014 and 2023.")


### Tables and Figures